The objective of the final data pipeline is parsing each column of the CSV file into the correct data type and save the new data as Parquet file.

# 1. `games_description.csv`

The file wraps all columns as a string. Attempt to parse the data type from the start did not work.

Schema:
```
name: string
short_description: string
long_description: string
genres: object (array[string])
minimum_system_requirement: object (struct[string])
recommend_system_requirement: object (struct[string])
release_date: date
developer: object (array[string])
publisher: object (array[string])
overall_player_rating: categorical
number_of_reviews_from_purchased_people: int32
number_of_english_reviews: int32
link: string
```

In [74]:
import polars as pl
from pathlib import Path
import re

local_dir = Path("/teamspace/studios/this_studio/Steam-RecSys/data_pipeline/data")


def parse_reviews(value):
    if "%" in value:
        # Extract percentage and total number
        match = re.search(r"(\d+)% of ([\d,]+)", value)
        if match:
            percentage = int(match.group(1))
            total = int(match.group(2).replace(",", ""))
            return int((percentage / 100) * total)
    else:
        # Extract the number directly
        match = re.search(r"\(([\d,]+)\)", value)
        if match:
            return int(match.group(1).replace(",", ""))


def parse_system_requirements(requirements_list):
    result = {}
    for item in requirements_list:
        if ":" in item:
            key, value = item.split(":")[:2]
            result[key.strip()] = value.strip()
    return result


df = pl.read_csv(local_dir / "games_description.csv")
df = df.with_columns(
    pl.col("genres").str.replace_many(["]", "'", "["], "").str.split(", "),
    pl.col("number_of_english_reviews").str.replace_all(",", "").cast(pl.Int32),
    pl.col(["minimum_system_requirement", "recommend_system_requirement"])
    .str.replace_many(["]", "'", "["], "")
    .str.split(", ")
    .map_elements(parse_system_requirements, return_dtype=pl.Object),
    pl.col(["developer", "publisher"])
    .str.replace_many(["]", "'", "["], "")
    .str.split(", "),
    pl.col("overall_player_rating").cast(pl.Categorical("lexical")),
    pl.when(pl.col("release_date").str.contains(r"\d{1,2} \w{3}, \d{4}"))
    .then(pl.col("release_date").str.to_date("%d %b, %Y", strict=False))
    .otherwise(pl.col("release_date").str.to_date("%b %Y", strict=False))
    .alias("release_date"),
    pl.col("number_of_reviews_from_purchased_people").map_elements(
        parse_reviews, return_dtype=pl.Int32
    ),
)

df

name,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link
str,str,str,list[str],object,object,date,list[str],list[str],cat,i32,i32,str
"""Black Myth: Wukong""","""Black Myth: Wukong is an actio…","""About This Game Black M…","[""Mythology"", ""Action RPG"", … ""Violent""]","{'OS': 'Windows 10 64-bit', 'Processor': 'Intel Core i5-8400 / AMD Ryzen 5 1600', 'Memory': '16 GB RAM', 'Graphics': 'NVIDIA GeForce GTX 1060 6GB / AMD Radeon RX 580 8GB', 'DirectX': 'Version 11', 'Storage': '130 GB available space', 'Sound Card': 'Windows Compatible Audio Device', 'Additional Notes': 'HDD Supported'}","{'OS': 'Windows 10 64-bit', 'Processor': 'Intel Core i7-9700 / AMD Ryzen 5 5500', 'Memory': '16 GB RAM', 'Graphics': 'NVIDIA GeForce RTX 2060 / AMD Radeon RX 5700 XT / INTEL Arc A750', 'DirectX': 'Version 12', 'Storage': '130 GB available space', 'Sound Card': 'Windows Compatible Audio Device', 'Additional Notes': 'SSD Required. The above specifications were tested with DLSS/FSR/XeSS enabled.'}",2024-08-19,"[""Game Science""]","[""Game Science""]","""Overwhelmingly Positive""",654820,51931,"""https://store.steampowered.com…"
"""Counter-Strike 2""","""For over two decades, Counter-…","""About This Game For ove…","[""FPS"", ""Shooter"", … ""Moddable""]","{'OS': 'Windows® 10', 'Processor': '4 hardware CPU threads - Intel® Core™ i5 750 or higher', 'Memory': '8 GB RAM', 'Graphics': 'Video card must be 1 GB or more and should be a DirectX 11-compatible with support for Shader Model 5.0', 'DirectX': 'Version 11', 'Storage': '85 GB available space'}","{'OS': 'Windows® 10', 'Processor': '4 hardware CPU threads - Intel® Core™ i5 750 or higher', 'Memory': '8 GB RAM', 'Graphics': 'Video card must be 1 GB or more and should be a DirectX 11-compatible with support for Shader Model 5.0', 'DirectX': 'Version 11', 'Storage': '85 GB available space'}",2012-08-21,"[""Valve""]","[""Valve""]","""Very Positive""",8313603,2258990,"""https://store.steampowered.com…"
"""Warhammer 40,000: Space Marine…","""Embody the superhuman skill an…","""About This Game Embody the sup…","[""Warhammer 40K"", ""Action"", … ""Futuristic""]","{'OS': 'Windows 10 (1903 min)/11 64-bit', 'Processor': 'AMD Ryzen 5 2600X / Intel Core i5-8600K', 'Memory': '8 GB RAM', 'Graphics': '6 GB VRAM', 'DirectX': 'Version 12', 'Storage': '75 GB available space', 'Additional Notes': 'SSD required. 30 FPS in 1920x1080 with ""Low"" preset.'}","{'OS': 'Windows 10 (1903 min)/11 64-bit', 'Processor': 'AMD Ryzen 7 5800X / Intel Core i7-12700', 'Memory': '16 GB RAM', 'Graphics': '8 GB VRAM', 'DirectX': 'Version 12', 'Storage': '75 GB available space', 'Additional Notes': 'SSD required. 60 FPS in 1920x1080 with the ""Ultra"" preset.'}",2024-09-09,"[""Saber Interactive""]","[""Focus Entertainment""]","""Very Positive""",50860,51920,"""https://store.steampowered.com…"
"""Cyberpunk 2077""","""Cyberpunk 2077 is an open-worl…","""About This Game Cyberpunk 2077…","[""Cyberpunk"", ""Open World"", … ""Immersive Sim""]","{'OS': '64-bit Windows 10', 'Processor': 'Core i7-6700 or Ryzen 5 1600', 'Memory': '12 GB RAM', 'Graphics': 'GeForce GTX 1060 6GB or Radeon RX 580 8GB or Arc A380', 'DirectX': 'Version 12', 'Storage': '70 GB available space', 'Additional Notes': 'SSD required. Attention'}","{'OS': '64-bit Windows 10', 'Processor': 'Core i7-12700 or Ryzen 7 7800X3D', 'Memory': '16 GB RAM', 'Graphics': 'GeForce RTX 2060 SUPER or Radeon RX 5700 XT or Arc A770', 'DirectX': 'Version 12', 'Storage': '70 GB available space', 'Additional Notes': 'SSD required.'}",2020-12-10,"[""CD PROJEKT RED""]","[""CD PROJEKT RED""]","""Very Positive""",680264,324124,"""https://store.steampowered.com…"
"""ELDEN RING""","""THE CRITICALLY ACCLAIMED FANTA…","""About This Game THE CRITICALLY…","[""Souls-like"", ""Dark Fantasy"", … ""Family Friendly""]","{'OS': 'Windows 10', 'Pr